# 🧰 Initial Settings

## 🚫 Warning Message Filter in `stderr` (TensorFlow/GPU)

In [ ]:

import sys, os

os.environ.update({
    'TF_CPP_MIN_LOG_LEVEL': '3',
    'TF_CPP_MIN_VLOG_LEVEL': '0',
    'GLOG_minloglevel': '3', 
    'GLOG_v': '0',
    'ABSL_LOG_SEVERITY': '3',
    'TF_GPU_THREAD_MODE': 'gpu_private',
    'TF_ENABLE_GPU_GARBAGE_COLLECTION': 'false'
})

class StderrFilter:
    def __init__(self):
        self.original_stderr = sys.stderr
        self.filtered_messages = [
            'Skipping the delay kernel',
            'measurement accuracy will be reduced',
        ]
    
    def write(self, message):
        if not any(filter_msg in message for filter_msg in self.filtered_messages):
            self.original_stderr.write(message)
    
    def flush(self):
        self.original_stderr.flush()

sys.stderr = StderrFilter()

## 📚 Libraries

In [2]:
import numpy as np, random, shutil, logging
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))

Python: 3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]
TensorFlow: 2.17.1
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 🗂️ Dataset Download with Kaggle

In [4]:
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

api = KaggleApi()
api.authenticate()

# Probar con un dataset
DATASET_SLUG = "karimabdulnabi/fruit-classification10-class"
api.dataset_download_files(DATASET_SLUG, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/karimabdulnabi/fruit-classification10-class


### 📂 Data Path Configuration

In [5]:
# use these lines in Google Colab
#SRC_DIR = "/content/MY_data"
#CONTENT_DIR = "/content"

#train_dir = "/content/train"
#test_dir  = "/content/test"
#predict_dir = "/content/predict"


SRC_DIR = "./MY_data"
CONTENT_DIR = "./"

train_dir = "./train"
test_dir  = "./test"
predict_dir = "./predict"


### 🔄 Directory Organization and Relocation

In [6]:
# Moves the dataset subfolders [train, test, predict] from the source directory (SRC_DIR) into the main /content directory (CONTENT_DIR).
splits = ["train", "test", "predict"]

for split in splits:
    src, dst = os.path.join(SRC_DIR, split), os.path.join(CONTENT_DIR, split)
    if os.path.isdir(src):
        shutil.rmtree(dst, ignore_errors=True)
        shutil.move(src, dst)
        print(f"✔️ {split} moved to {dst}")
    else:
        print(f"⚠️ {split} not found in {SRC_DIR}")

shutil.rmtree(SRC_DIR, ignore_errors=True)


✔️ train moved to ./train
✔️ test moved to ./test
✔️ predict moved to ./predict


# 🧠 Design and Training of a Custom CNN

In [7]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import (Input, Conv2D, MaxPooling2D, Dense, Flatten,
                                     Dropout, RandomRotation, RandomTranslation,
                                     RandomFlip, Rescaling, GlobalAveragePooling2D)
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


## ⚙️ Training Hyperparameters

In [8]:
IMG_SIZE=(224,224)
BATCH_SIZE=32
SEED=42
EPOCHS = 20

## 📂 Dataset Loading and Splitting (Train/Validation/Test)


In [9]:
print("📂 Loading training dataset...")
train_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    label_mode="categorical"
)

print("\n📂 Loading validation dataset...")
val_ds = keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    label_mode="categorical"
)

print("\n📂 Loading test dataset...")
test_ds = keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    label_mode="categorical"
)


📂 Loading training dataset...
Found 2301 files belonging to 10 classes.
Using 1841 files for training.

📂 Loading validation dataset...
Found 2301 files belonging to 10 classes.
Using 460 files for validation.

📂 Loading test dataset...
Found 1025 files belonging to 10 classes.


### 🏷️ Dataset Classes

In [10]:
class_names = train_ds.class_names
num_classes = len(class_names)

print("class_names:", class_names)
print("num_classes:", num_classes)

class_names: ['Apple', 'Banana', 'avocado', 'cherry', 'kiwi', 'mango', 'orange', 'pinenapple', 'strawberries', 'watermelon']
num_classes: 10


## 🎛️ Data Augmentation and Normalization Layers

In [11]:
data_augmentation = Sequential([
    RandomRotation(0.11),         # rotation_range=40
    RandomTranslation(0.1, 0.1),  # width_shift & height_shift
    RandomFlip("horizontal"),     # horizontal_flip
], name="data_augmentation")

normalization = Rescaling(1./255, name="rescale")


## 🏗️ Build and Compile Custom CNN Model

In [12]:
def build_custom_cnn():
  model = Sequential([
      Input(shape=IMG_SIZE+(3,)),
      data_augmentation,
      normalization,

      Conv2D(32, 3, padding="same", activation="relu"),
      Conv2D(32, 3, padding="same", activation="relu"),
      MaxPooling2D(),

      Conv2D(64, 3, padding="same", activation="relu"),
      Conv2D(64, 3, padding="same", activation="relu"),
      MaxPooling2D(),

      Conv2D(128, 3, padding="same", activation="relu"),
      MaxPooling2D(),

      Flatten(),
      Dense(256, activation="relu"),
      Dropout(0.5),
      Dense(num_classes, activation="softmax")
  ], name="CustomCNN")

  model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
  
  return model


In [13]:
custom_model = build_custom_cnn()
custom_model.summary()

Model: "CustomCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescale (Rescaling)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,832,362 (98.54 MB)

 Trainable params: 25,832,362 (98.54 MB)

 Non-trainable params: 0 (0.00 B)

## ⏱️ Training Custom CNN

In [14]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor="val_loss"),
    ModelCheckpoint("custom_cnn.keras", monitor="accuracy", save_best_only=True),
    ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_loss")
]

hist_custom = custom_model.fit(
    train_ds, 
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks,
    )

Epoch 1/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 15s 145ms/step - accuracy: 0.1744 - loss: 2.1973 - val_accuracy: 0.2761 - val_loss: 1.8778 - learning_rate: 0.0010
Epoch 2/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 8s 145ms/step - accuracy: 0.3107 - loss: 1.7576 - val_accuracy: 0.3478 - val_loss: 1.7887 - learning_rate: 0.0010
Epoch 3/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - accuracy: 0.3313 - loss: 1.6996 - val_accuracy: 0.3739 - val_loss: 1.7258 - learning_rate: 0.0010
Epoch 4/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - accuracy: 0.3721 - loss: 1.6041 - val_accuracy: 0.4413 - val_loss: 1.5281 - learning_rate: 0.0010
Epoch 5/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 106ms/step - accuracy: 0.3922 - loss: 1.5724 - val_accuracy: 0.4391 - val_loss: 1.4880 - learning_rate: 0.0010
Epoch 6/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - accuracy: 0.3987 - loss: 1.5451 - val_accuracy: 0.4413 - val_loss: 1.5664 - learning_rate: 0.0010
Epoch 7/20
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - accuracy: 0.4221 - loss: 1.4790 -

### 🧪 Model Evaluation on Test Set

In [15]:
test_loss, test_acc = custom_model.evaluate(test_ds)
print(f"Precisión en test: {test_acc:.4f}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5151 - loss: 1.5599
Precisión en test: 0.5151


# 🔄 Transfer Learning with Pre-trained CNN

In [16]:
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input

## 🧩 Build MobileNetV2 Transfer-Learning Head

In [17]:
def build_mobilenet_head(base_trainable=False, fine_tune_at=None):
    MobileNetV2_base  = MobileNetV2(
        input_shape=IMG_SIZE+(3,),
        include_top=False,
        weights="imagenet",
        pooling="avg",
        )

    MobileNetV2_base.trainable = base_trainable
    
    if fine_tune_at is not None:
      for layer in MobileNetV2_base.layers[:fine_tune_at]:
          layer.trainable = False

    base_learning_rate = 1e-3
    inputs = Input(shape=IMG_SIZE+(3,))
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    x = MobileNetV2_base(x, training=False)
    x = Dropout(0.3)(x)
    x = Dense(220, activation="relu")(x)
    x = Dense(60, activation="relu")(x)
    outputs = Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs, name="MobileNetV2_TL")
    model.compile(optimizer=tf.keras.optimizers.Adam(base_learning_rate),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model, MobileNetV2_base



In [18]:
tl_model, MobileNetV2_base = build_mobilenet_head()
tl_model.summary()

Model: "MobileNetV2_TL"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 220)            │       281,820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 60)             │        13,260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,553,674 (9.74 MB)

 Trainable params: 295,690 (1.13 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

## ⏱️ Training MobileNetV2 (Feature Extraction Phase)

In [19]:
callbacks = [EarlyStopping(patience=3, restore_best_weights=True, monitor="val_loss")]

hist_tl_1 = tl_model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=8,
    callbacks=callbacks
)

Epoch 1/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 9s 80ms/step - accuracy: 0.6931 - loss: 0.9263 - val_accuracy: 0.8804 - val_loss: 0.3200
Epoch 2/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.8707 - loss: 0.3962 - val_accuracy: 0.9239 - val_loss: 0.2362
Epoch 3/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.9098 - loss: 0.2787 - val_accuracy: 0.9130 - val_loss: 0.2327
Epoch 4/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - accuracy: 0.9131 - loss: 0.2715 - val_accuracy: 0.9130 - val_loss: 0.2211
Epoch 5/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.9310 - loss: 0.1932 - val_accuracy: 0.9304 - val_loss: 0.2071
Epoch 6/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.9495 - loss: 0.1515 - val_accuracy: 0.9326 - val_loss: 0.1966
Epoch 7/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.9430 - loss: 0.1718 - val_accuracy: 0.9304 - val_loss: 0.2356
Epoch 8/8
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.9538 - loss: 0.1454 - val_accuracy: 0.9304 - val_loss

### 🔧 Initialize Fine-Tuned MobileNetV2 with Pretrained Weights

In [20]:
fine_tune_at = int(0.75 * len(MobileNetV2_base.layers))
tl_model_ft, _ = build_mobilenet_head(base_trainable=True, fine_tune_at=fine_tune_at)

tl_model_ft.set_weights(tl_model.get_weights())

tl_model_ft.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                    loss="categorical_crossentropy", metrics=["accuracy"])

## 🚀 MobileNetV2 Training (Fine-Tuning Phase)

In [21]:
callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True, monitor="val_loss"),
        ReduceLROnPlateau(patience=2, factor=0.5, monitor="val_loss"),
        ModelCheckpoint("mobilenetv2_tl.keras", monitor="val_accuracy", save_best_only=True)
]

hist_tl_2 = tl_model_ft.fit(
    train_ds, validation_data=val_ds, epochs=50,
    callbacks=callbacks
)

Epoch 1/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 16s 135ms/step - accuracy: 0.8658 - loss: 0.4161 - val_accuracy: 0.8739 - val_loss: 0.4215 - learning_rate: 1.0000e-04
Epoch 2/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 8s 138ms/step - accuracy: 0.9310 - loss: 0.2029 - val_accuracy: 0.8957 - val_loss: 0.3436 - learning_rate: 1.0000e-04
Epoch 3/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.9500 - loss: 0.1500 - val_accuracy: 0.8870 - val_loss: 0.4270 - learning_rate: 1.0000e-04
Epoch 4/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - accuracy: 0.9696 - loss: 0.0961 - val_accuracy: 0.8739 - val_loss: 0.4982 - learning_rate: 1.0000e-04
Epoch 5/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - accuracy: 0.9674 - loss: 0.1025 - val_accuracy: 0.8565 - val_loss: 0.5268 - learning_rate: 5.0000e-05
Epoch 6/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.9810 - loss: 0.0603 - val_accuracy: 0.8804 - val_loss: 0.4572 - learning_rate: 5.0000e-05
Epoch 7/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.97

### 🧪 Fine-Tuned Model Test Evaluation

In [22]:
test_loss, test_acc = tl_model_ft.evaluate(test_ds)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.7337 - loss: 3.3109
